In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px

import warnings
warnings.filterwarnings("ignore")


In [2]:
file_path = 'processed_dataset.csv' 
df= pd.read_csv(file_path)

In [3]:
X= df.drop(columns=['Recycling Rate (%)'])
y= df['Recycling Rate (%)']

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score,root_mean_squared_error


In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train

Waste Type  Waste Generated (Tons/Day)  ...  Year  city
332           3                        8893  ...  2020    14
383           4                         917  ...  2020    16
281           2                        5690  ...  2020    12
2             3                        8162  ...  2019     1
231           2                         974  ...  2020    10
..          ...                         ...  ...   ...   ...
71            2                        7135  ...  2023     3
106           2                        6472  ...  2020     5
270           1                        7045  ...  2023    11
435           1                        7724  ...  2021    18
102           3                         812  ...  2019     5

[680 rows x 10 columns]

In [16]:
X_train.shape

(680, 10)

In [17]:
X_test.shape

(170, 10)

In [21]:
model={"LinearRegression":LinearRegression(),
       "RandomForestRegressor":RandomForestRegressor(), 
       "Tree":DecisionTreeRegressor(),
       }

In [22]:
# model training for each model using for loop fro name, model in model.items():
# for best model selection:
for name, model in model.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)*100  
    root_mse = np.sqrt(mse)
    
    print(f"{name} - MSE: {mse}, R2: {r2} , Root MSE: {root_mse}")

LinearRegression - MSE: 277.79130538548606, R2: -2.1241026725161705 , Root MSE: 16.66707248995714
RandomForestRegressor - MSE: 302.4142929411765, R2: -11.176223673034702 , Root MSE: 17.390063051673405
Tree - MSE: 575.4529411764706, R2: -111.5531123854165 , Root MSE: 23.98860023378752


In [23]:
mod=XGBRegressor(n_estimators=100, learning_rate=0.1,random_state=42)

In [25]:
mod.fit(X_train, y_train)
y_pred = mod.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)*100
root_mse = np.sqrt(mse)

In [26]:
mse, r2, root_mse

(350.62896728515625, -28.901326656341553, np.float64(18.72508924638695))

In [27]:
cat_mod=CatBoostRegressor(iterations=100, learning_rate=0.1, random_state=42, verbose=0)
cat_mod.fit(X_train, y_train)
y_pred = cat_mod.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)*100
root_mse = np.sqrt(mse)

In [28]:
mse, r2, root_mse

(297.53285871589463, -9.381667542776984, np.float64(17.2491408109475))

# from mse score we can use CatBoost Regressor for the machine learning model 

# now performing the hyperparameter tuning for catboost regressor using grid search CV

In [29]:
params = {
    'iterations': [100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8], 
    'l2_leaf_reg': [1, 3, 5]
}

In [30]:
hy_model= GridSearchCV(estimator=cat_mod, param_grid=params, cv=3, scoring='neg_mean_squared_error', verbose=1)
hy_model.fit(X_train, y_train)  

Fitting 3 folds for each of 54 candidates, totalling 162 fits


GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostRegressor object at 0x000002B2C78868D0>,
             param_grid={'depth': [4, 6, 8], 'iterations': [100, 200],
                         'l2_leaf_reg': [1, 3, 5],
                         'learning_rate': [0.01, 0.1, 0.2]},
             scoring='neg_mean_squared_error', verbose=1)

In [31]:
best_params = hy_model.best_params_
best_params

{'depth': 4, 'iterations': 100, 'l2_leaf_reg': 5, 'learning_rate': 0.01}